In [68]:
import pandas as pd
import os
import re

annotations = pd.read_csv("DLCV_logo_project/annot_train.csv")
annotations['class'] = annotations['class'].apply(lambda x: re.sub(' ', '_', x))
annotations['class'] = annotations['class'].apply(lambda x: re.sub('-', '_', x))
annotations.head()

,photo_filename,width,height,class,xmin,ymin,xmax,ymax
0,charlotte_1019088214340003257_20150630.jpg,640,640,Nike,339,394,381,439
1,charlotte_1056018576661922536_20150820.jpg,640,640,Nike,497,0,574,30
2,charlotte_1057610050380125945_20150823.jpg,1080,1080,Nike,353,406,473,477
3,charlotte_1061715143228072770_20150828.jpg,1080,1349,Nike,13,51,330,198
4,charlotte_1081463278280976819_20150925.jpg,612,612,Nike,485,30,580,69


In [72]:
# annotations['class'].unique()
annotations.to_csv('DLCV_logo_project/annotations.csv', index=False)

In [64]:
mandatory_classes = 'Nike, Adidas, Under_Armour, Puma, The_North_Face'.split(", ")
annotations_mandatory = annotations[annotations['class'].apply(lambda x: x in mandatory_classes)]
annotations_optional = annotations[annotations['class'].apply(lambda x: x not in mandatory_classes)]
print(f'Mandatory annotations: {len(annotations_mandatory)}\nOptional annotations: {len(annotations_optional)}')

annotations_mandatory.to_csv('annotations_mandatory.csv')
annotations_optional.to_csv('annotations_optional.csv')

Mandatory annotations: 21675
Optional annotations: 24488


<ipython-input-64-c57fb8cbf5be>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotations_optional['class'] = annotations_optional['class'].apply(lambda x: re.sub(' ', '_', x))
<ipython-input-64-c57fb8cbf5be>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotations_optional['class'] = annotations_optional['class'].apply(lambda x: re.sub('-', '_', x))


In [5]:
train_data_dir = os.path.join(os.getcwd(), 'DLCV_logo_project/train')
train_pics = os.listdir(train_data_dir)

# moving mandatory 5 logos to train/mandatory_logos folder
mandatory_pics = [pic for pic in train_pics if pic in annotations_mandatory['photo_filename'].tolist()]
os.mkdir(os.path.join(os.getcwd(), 'DLCV_logo_project/train/mandatory_logos'))
mandatory_logos_dir = os.path.join(os.getcwd(), 'DLCV_logo_project/train/mandatory_logos')           
for pic in mandatory_pics:
    os.rename(os.path.join(train_data_dir, pic), os.path.join(mandatory_logos_dir, pic))

In [20]:
# moving optional 12 logos to train/mandatory_logos folder
optional_logos_dir = os.path.join(os.getcwd(), 'DLCV_logo_project/train/optional_logos')   
os.mkdir(optional_logos_dir)
optional_pics = [pic for pic in train_pics if pic not in annotations_mandatory['photo_filename'].tolist()]
for pic in optional_pics:
    os.rename(os.path.join(train_data_dir, pic), os.path.join(optional_logos_dir, pic))

In [26]:
print(f'Number of mandatory 5 classes train pics: {len(os.listdir(mandatory_logos_dir))}\
      \nNumber of optional 12 classes train pics: {len(os.listdir(optional_logos_dir))}')

Number of mandatory 5 classes train pics: 20974      
Number of optional 12 classes train pics: 17939


In [42]:
# splitting training in batches for Roboflow to process

batch1_dir = os.path.join(os.getcwd(),'DLCV_logo_project/train/mandatory_logos/batch1')
batch2_dir = os.path.join(os.getcwd(), 'DLCV_logo_project/train/mandatory_logos/batch2')
batch3_dir = os.path.join(os.getcwd(),'DLCV_logo_project/train/mandatory_logos/batch3')
os.mkdir(batch1_dir)
os.mkdir(batch2_dir)
os.mkdir(batch3_dir)

for pic in [pic for pic in os.listdir(mandatory_logos_dir) if pic.endswith('jpg')][:7000]: # first 7000 pics
    os.rename(os.path.join(mandatory_logos_dir, pic), os.path.join(batch1_dir, pic))
for pic in [pic for pic in os.listdir(mandatory_logos_dir) if pic.endswith('jpg')][:7000]: # second 7000 pics
    os.rename(os.path.join(mandatory_logos_dir, pic), os.path.join(batch2_dir, pic))
for pic in [pic for pic in os.listdir(mandatory_logos_dir) if pic.endswith('jpg')]: # the rest
    os.rename(os.path.join(mandatory_logos_dir, pic), os.path.join(batch3_dir, pic))

In [43]:
print(f'Batch 1 size: {len(os.listdir(batch1_dir))}\
      \nBatch 2 size: {len(os.listdir(batch2_dir))}\
      \nBatch 3 size: {len(os.listdir(batch3_dir))}')

Batch 1 size: 7645      
Batch 2 size: 7000      
Batch 3 size: 6329
